# Pre-processing of Lagain Mars craters for DaCHS DB ingestion

To ingest data from Lagain & Chiara data release -- https://github.com/alagain/martian_crater_database -- into DaCHS, which is a Postgres db, we must first workout the GeoJSON files provided. We should arrive in a flat table.

The workflow is basically:
* read JSON
 * unravel data in one or more tables
* check data
* write CSV

In [1]:
import json
import pandas
print('Pandas version: ', pandas.__version__)

Pandas version:  0.24.1


<div class="alert alert-info">
Where are we?

<pre>
<code>
$ git clone https://github.com/alagain/martian_crater_database.git
$ cd martian_crater_database/Global
$ unzip lagain_db.json.zip
</code>
</pre>
</div>

In [2]:
%ls

lagain_db_filtered.json.zip  Notebook_PreProcLagain.html
lagain_db_filtered.zip       Notebook_PreProcLagain.ipynb
lagain_db.json               notebook.tex
lagain_db.json.zip           README.md
lagain_db.zip


In [3]:
with open('lagain_db.json','r') as fp:
    js = json.load(fp)
    
features = js['features']
print("Number of features: ", len(features))

Number of features:  384582


In [4]:
features[0]

{'type': 'Feature',
 'properties': {'CRATER_ID': '200-007',
  'RADIUS': 500.0,
  'X': 23.671499,
  'Y': -43.584301,
  'TYPE': 1.0,
  'STATUS': 'Valid',
  'LRD_MORPH': None,
  'ORIGIN': None,
  'ADDING': 1.0},
 'geometry': {'type': 'Point', 'coordinates': [23.671499, -43.584301]}}

## Let's check if everything is "Point"
...sanity check actually, because asaik they are all points...

In [5]:
set([f['geometry']['type'] for f in features])

{'Point'}

...all entries are points. OK. Which, together with our "first feature" sample above, I assume that (`X`,`Y`) and `coordinates` have the same values. That being the case, I can simply drop `geometry` in what follows.

## Let's check what is going on in `properties`

In [6]:
df = pandas.read_json(json.dumps([f['properties'] for f in features]))

In [7]:
df.describe(include='all')

,ADDING,CRATER_ID,LRD_MORPH,ORIGIN,RADIUS,STATUS,TYPE,X,Y
count,384582.000000,384582,8480,39204,384582.000000,384582,384582.000000,384582.000000,384582.000000
unique,NaN,384561,4,106,NaN,5,NaN,NaN,NaN
top,NaN,21-003453,SLE,05-000000,NaN,Valid,NaN,NaN,NaN
freq,NaN,2,6680,5857,NaN,288117,NaN,NaN,NaN
mean,0.000793,NaN,NaN,NaN,1778.392579,NaN,1.665770,10.135017,-7.180770
std,0.028150,NaN,NaN,NaN,4295.514407,NaN,1.211242,96.634768,33.612876
min,0.000000,NaN,NaN,NaN,500.000000,NaN,1.000000,-179.996994,-86.699997
25%,0.000000,NaN,NaN,NaN,590.000000,NaN,1.000000,-58.806000,-30.928750
50%,0.000000,NaN,NaN,NaN,765.000000,NaN,1.000000,12.759500,-10.056000
75%,0.000000,NaN,NaN,NaN,1280.000000,NaN,2.000000,89.264749,17.259001


In [8]:
pandas.set_option('display.max_rows',100)
df.sample(100)

,ADDING,CRATER_ID,LRD_MORPH,ORIGIN,RADIUS,STATUS,TYPE,X,Y
280879,0,24-001127,None,None,6895,Valid,1,-174.186005,-58.084999
45563,0,20-015722,None,None,550,Valid,1,5.905000,-26.218000
213574,0,07-001750,None,None,1115,Valid,1,152.475006,55.555000
24490,0,24-014612,None,None,525,Valid,1,-151.048996,-38.698002
131435,0,12-012470,None,None,695,Valid,1,14.999000,24.162001
3314,0,18-014319,None,None,505,Valid,1,-76.845001,-21.645000
359626,0,26-011110,None,26-000008,590,Secondary,4,-15.291000,-48.409000
268618,0,02-000382,None,None,2900,Valid,1,-155.679001,39.369999
228086,0,02-001071,None,None,1290,Valid,1,-151.828995,38.723999
294720,0,21-001286,SLE,None,5475,Layered,2,63.188999,-17.815001
